# sklearn_Clustering - 군집분석



# 실습용 데이터 설정

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
from sklearn.datasets import load_iris

iris = load_iris()

In [4]:
DF=pd.DataFrame(data = iris.data,
                columns = ['sepal_length','sepal_width','petal_length','petal_width'])
DF.head(3)

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [5]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB


# 1, K-means

## 1) Modeling

In [6]:
from sklearn.cluster import KMeans

kmeans_3 = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 15, random_state= 2045)
kmeans_3.fit(iris.data) 

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=15,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=2045, tol=0.0001, verbose=0)

## 2) 군집결과

In [7]:
DF['kmeans'] = kmeans_3.labels_
DF['target'] = iris.target

DF.groupby('target')['kmeans'].value_counts()

target  kmeans
0       1         50
1       0         48
        2          2
2       2         36
        0         14
Name: kmeans, dtype: int64

In [8]:
DF.head()

,sepal_length,sepal_width,petal_length,petal_width,kmeans,target
0,5.1,3.5,1.4,0.2,1,0
1,4.9,3.0,1.4,0.2,1,0
2,4.7,3.2,1.3,0.2,1,0
3,4.6,3.1,1.5,0.2,1,0
4,5.0,3.6,1.4,0.2,1,0


# 2, Mean Shift(평균 이동) - 밀도
* 데이터의 분포를 이용하여 군집의 중심을 탐색
  * 데이터 밀도가 가장 높은 곳으로 중심을 지속적으로 이동
* 군집의 중심점은 데이터포인트가 모여있는 곳이라는 가정
  * 확률 밀도 함수(Probability Density Function)
  * KDF(Kernel Density Estimation)

## 1) Modeling
* Hyperparameter
  * bandwidth : 대역폭이 클수록 적은 수의 군집 중심을 가짐
  

In [9]:
from sklearn.cluster import MeanShift

meanshift = MeanShift(bandwidth = 0.86) # bandwidth = 0.86은 되야 군집에 3개로 나와.. 0.2하면 군집이 엄청나게 많아짐, 1.2는 2개로 나눠짐
meanshift.fit(iris.data)  

MeanShift(bandwidth=0.86, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [10]:
from sklearn.cluster import estimate_bandwidth

# bandwidth에 아무런 정보없이 아무런 수나 넣을 수 없으니까 제시해주는 값
estimate_bandwidth(iris.data)
# 1.2로 하면 2개로 묶여짐. 그게 최선의 방법이라고 제시

1.2020768127998687

## 2) 군집결과

In [11]:
DF['meanshift'] = meanshift.fit_predict(iris.data)
DF.groupby('target')['meanshift'].value_counts()
# 1,2에 차이가 미세하니까 1을 0으로 하거나 1을 2로 하거나 해서 2개의 군집으로 묶으라고 제시

target  meanshift
0       1            50
1       0            48
        1             1
        2             1
2       2            36
        0            14
Name: meanshift, dtype: int64

# 3, GMM(Gaussian Mixture Model)
* 데이터가 여러 개의 가우시안 분포(Gaussian Distribution)를 가진 데이터들의 집합이라고 가정
  * 개별 정규분포의 모수(평균, 분산) 추정
  * 각 데이터포인트가 어떤 정규분포에 해당하는지 확률 추정
    * EM(Expectation and Maximization)

## 1) Modeling

In [12]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components = 3,  # 몇개의 정규분포로 되어 있을 거라는/ 가우시안 분포가 3개의 component(정규식)의 합
                      random_state= 2045)
gmm.fit(iris.data)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=3, n_init=1, precisions_init=None,
                random_state=2045, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [13]:
gmm.predict(iris.data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## 2) 군집 결과

In [14]:
DF['gmm'] = gmm.predict(iris.data)
DF.groupby('target')['gmm'].value_counts()

target  gmm
0       1      50
1       0      45
        2       5
2       2      50
Name: gmm, dtype: int64

In [15]:
DF.gmm.value_counts()

2    55
1    50
0    45
Name: gmm, dtype: int64

# 4, DBSCAN(Density Based Spatial Clustering of Applications with Noise)
* 밀도(Density) 기반 군집
  * 기하학적으로 복잡한 데이터에도 효과적으로 군집 가능
  * 핵심 포인트(Core Point)들을 서로 연결하면서 군집화
* 밀도의 기준을 어떻게 볼 것인가? - 문제

## 1) Modeling
* Hyperparameter
  * epsilon(esp, 입실론 주변 영역) # 입실론 = 그리스자
    * 개별 데이터포인트를 중심으로 '입실론 반경'을 가지는 주변 영역
    * 'Core Point' 기준
  * min point(min_samples, 최소 데이터 개수)
    * 개별 데이터포인트의 '입실론 주변 영역'에 포함되는 다른 데이터포인트의 개수
    * 조건 만족 시 'Core Point'로 지정

In [16]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.8, # 0.8 반지름으로/ 3으로 했을 때는 하나로 묶어버림. 반지름이 너무 커서
                min_samples =8, # 중심이 되는 데이터포인트(core point)를 제외하고 최소 8개가 들어있는
                metric= 'euclidean') # euclidean = 직선거리로 측정

dbscan.fit_predict(iris.data)
# -1은 비정상(?)적 값. 

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

## 2) 군집 결과

In [17]:
DF['dbscan'] = dbscan.fit_predict(iris.data)
DF.groupby('target')['dbscan'].value_counts()

target  dbscan
0        0        50
1        1        50
2        1        47
        -1         3
Name: dbscan, dtype: int64